In [1]:
from keras import models
from keras import layers
from keras.utils import np_utils, generic_utils
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG19
from sklearn.model_selection import KFold
from keras.optimizers import adam
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
### Get data from the input file ###

infile = open('S:\\Usyd\\Project\\Data\\train_data','rb')
Images = pickle.load(infile)
Labels = pickle.load(infile)
Patientid = pickle.load(infile)
infile.close()

In [3]:
### Data ###

# Convert labels into binary #
binaryLabels = np_utils.to_categorical(Labels)

# Training data #
xtrain = Images
ytrain = binaryLabels
print(xtrain.shape)
print(ytrain.shape)

In [4]:
#create array with only negative findings

xtrain_neg = []
ytrain_neg = []
for i in range(len(xtrain)):
    if ytrain[i,1]==1:
        xneg = xtrain[i,:,:,:]
        yneg = ytrain[i,:]
        xtrain_neg.append(xneg)
        ytrain_neg.append(yneg)
        
import numpy as np
xtrain_neg = np.array(xtrain_neg)
ytrain_neg = np.array(ytrain_neg)

In [5]:
#create array with only positive findings

xtrain_pos = []
ytrain_pos = []
for i in range(len(xtrain)):
    if ytrain[i,0]==1:
        xpos = xtrain[i,:,:,:]
        ypos = ytrain[i,:]
        xtrain_pos.append(xpos)
        ytrain_pos.append(ypos)
        
xtrain_pos = np.array(xtrain_pos)
ytrain_pos = np.array(ytrain_pos)

In [6]:
#Check the dimension of arrays
print(xtrain_neg.shape)
print(ytrain_neg.shape)
print(xtrain_pos.shape)
print(ytrain_pos.shape)

(47, 224, 224, 3)
(47, 2)
(149, 224, 224, 3)
(149, 2)


In [7]:
# Create a datagen for transformation
datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
g = datagen.flow(xtrain_pos, ytrain_pos, batch_size=1)
gg = datagen.flow(xtrain_neg, ytrain_neg, batch_size=1)

In [8]:
# Create variables to hold the transformed data
# One for X, one for Y
trDataX = []
trDataY = []

In [9]:
# Run the generator multiples times
N = len(xtrain_pos) * 13
for i in range(N):
  xBatch, yBatch = next(g)
  trDataX.append(xBatch)
  trDataY.append(yBatch)

In [10]:
M = len(xtrain_neg) * 40
for i in range(M):
  xBatch, yBatch = next(gg)
  trDataX.append(xBatch)
  trDataY.append(yBatch)

In [11]:
# Convert the list to np.array
# np.concatenate will merge the data along an axis
import numpy as np
trDataX = np.concatenate(trDataX, axis=0)
trDataY = np.concatenate(trDataY, axis=0)

In [12]:
# Check the dimension
print(trDataX.shape)
print(trDataY.shape)

(3817, 224, 224, 3)
(3817, 2)


In [18]:
# Check the corresponding label
# Start from 0
from sklearn.utils import shuffle
trDataX, trDataY = shuffle(trDataX, trDataY, random_state=0)

In [16]:
with open('S:\\Usyd\\Project\\Data\\train_data_balanced', 'wb') as f:
    pickle.dump(trDataX, f)
    pickle.dump(trDataY, f)